In [1]:
import json
import requests
import pandas as pd

In [24]:
def data_caller(url, headers):
    response = requests.get(url, headers)
    return response

def get_json(url):
    r = requests.get(url)
    return r.json()

def dim_table():
    # create the dim table
    dim_json = []
    regiones = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/regiones/all.json')
    for region in regiones:
        circs_senatorial = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/circ_senatorial/byregion/{}.json'.format(region['c']))
        for circ_senatorial in circs_senatorial:
            distritos = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/distritos/bycirc_senatorial/{}.json'.format(circ_senatorial['c']))
            for distrito in distritos:
                comunas = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/comunas/bydistrito/{}.json'.format(distrito['c']))
                for comuna in comunas:
                    circs_electoral = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/circ_electoral/bycomuna/{}.json'.format(comuna['c']))
                    for circ_electoral in circs_electoral:
                        locales = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/locales/bycirc_electoral/{}.json'.format(circ_electoral['c']))
                        for local in locales:
                            mesas = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/mesas/bylocales/{}.json'.format(local['c']))
                            for mesa in mesas:
                                row = {
                                    'region_id': region['c'],
                                    'region_name': region['d'],
                                    'circ_senatorial_id': circ_senatorial['c'],
                                    'circ_senatorial_name': circ_senatorial['d'],
                                    'distrito_id': distrito['c'],
                                    'distrito_name': distrito['d'],
                                    'comuna_id': comuna['c'],
                                    'comuna_name': comuna['d'],
                                    'circ_electoral_id': circ_electoral['c'],
                                    'circ_electoral_name': circ_electoral['d'],
                                    'local_id': local['c'],
                                    'local_name': local['d'],
                                    'mesa_id': mesa['c'],
                                    'mesa_name': mesa['d'],
                                }
                                dim_json.append(row)
    return dim_json

def dim_geo_servel():
    dim_json = []
    # create the dimension table for 
    regiones = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/regiones/all.json')
    for region in regiones:
        provincias = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/provincias/byregion/{}.json'.format(region['c']))
        for provincia in provincias:
            comunas = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/filters/comunas/byprovincia/{}.json'.format(provincia['c']))
            for comuna in comunas:
                row = {
                    'region_id': region['c'],
                    'region_name': region['d'],
                    'provincia_id': provincia['c'],
                    'provincia_name': provincia['d'],
                    'comuna_id': comuna['c'],
                    'comuna_name': comuna['d']
                }
                dim_json.append(row)
    return dim_json

def dict_results(data, name, optionA, optionB):
    dict = {
        '{}_{}'.format(name, optionA): data['data'][0]['c'],
        '{}_{}'.format(name, optionB): data['data'][1]['c'],
        '{}_nulos'.format(name): data['resumen'][1]['c'],
        '{}_blancos'.format(name): data['resumen'][2]['c'],
        '{}_total'.format(name): data['resumen'][3]['c'],
        '{}_validos'.format(name): data['resumen'][0]['c']
    }
    return dict

In [25]:
dim_geo_json = dim_geo_servel()
dim_geo = pd.DataFrame(dim_geo_json)
dim_geo.to_csv('dim_geo.csv', index=False)


In [26]:
dim_geo

,region_id,region_name,provincia_id,provincia_name,comuna_id,comuna_name
0,3015,DE ARICA Y PARINACOTA,4151,ARICA,2822,ARICA
1,3015,DE ARICA Y PARINACOTA,4151,ARICA,2823,CAMARONES
2,3015,DE ARICA Y PARINACOTA,4152,PARINACOTA,2825,GENERAL LAGOS
3,3015,DE ARICA Y PARINACOTA,4152,PARINACOTA,2824,PUTRE
4,3001,DE TARAPACA,4014,DEL TAMARUGAL,2504,CAMIÑA
...,...,...,...,...,...,...
341,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,4123,TIERRA DEL FUEGO,2753,PORVENIR
342,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,4123,TIERRA DEL FUEGO,2754,PRIMAVERA
343,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,4123,TIERRA DEL FUEGO,2755,TIMAUKEL
344,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,4124,ULTIMA ESPERANZA,2756,NATALES


In [29]:
# read dim_table
df = pd.read_csv('dim_electorales.csv')
df.merge(dim_geo[['provincia_id', 'provincia_name', 'comuna_id']], on='comuna_id')

,region_id,region_name,circ_senatorial_id,circ_senatorial_name,distrito_id,distrito_name,comuna_id,comuna_name,circ_electoral_id,circ_electoral_name,local_id,local_name,mesa_id,mesa_name,provincia_id,provincia_name
0,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,2985,COLEGIO DEL ALBA,70011235,235V-236V,4151,ARICA
1,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,2985,COLEGIO DEL ALBA,70011237,237V-238V,4151,ARICA
2,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,2985,COLEGIO DEL ALBA,70011239,239V-240V,4151,ARICA
3,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,2985,COLEGIO DEL ALBA,70011241,241V-242V,4151,ARICA
4,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,2985,COLEGIO DEL ALBA,70011243,243V-244V,4151,ARICA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44692,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),1313,ESCUELA RAMON SERRANO,75040001,1M-3M,4124,ULTIMA ESPERANZA
44693,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),1313,ESCUELA RAMON SERRANO,75040002,2M-1V,4124,ULTIMA ESPERANZA
44694,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),1313,ESCUELA RAMON SERRANO,75041002,2V-3V,4124,ULTIMA ESPERANZA
44695,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),1313,ESCUELA RAMON SERRANO,75041004,4V-5V,4124,ULTIMA ESPERANZA


In [76]:
with open('dim_electorales.json') as json_file:
    data = json.load(json_file)

In [ ]:
for row in data:
    if row['region_id'] == 3015:
        # votacion de constitucion
        constitucion_json = get_json('http://www.servelelecciones.cl/data/elecciones_constitucion/computomesas/{}.json'.format(row['mesa_id']))
        constitucion = dict_results(constitucion_json, 'constitucion', 'apruebo', 'rechazo')
        
        # votacion de tipo de organo
        convencion_json = get_json('http://www.servelelecciones.cl/data/elecciones_convencion/computomesas/{}.json'.format(row['mesa_id']))
        convencion = dict_results(convencion_json, 'convencion', 'mixta', 'constitucional')
        row.update(constitucion)
        row.update(convencion)

In [75]:
pd.DataFrame(data)

,region_id,region_name,circ_senatorial_id,circ_senatorial_name,distrito_id,distrito_name,comuna_id,comuna_name,circ_electoral_id,circ_electoral_name,...,constitucion_nulos,constitucion_blancos,constitucion_total,constitucion_validos,convencion_mixta,convencion_constitucional,convencion_nulos,convencion_blancos,convencion_total,convencion_validos
0,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44692,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44693,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44694,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44695,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = pd.read_csv('resultados.csv')
df

,region_id,region_name,circ_senatorial_id,circ_senatorial_name,distrito_id,distrito_name,comuna_id,comuna_name,circ_electoral_id,circ_electoral_name,...,constitucion_nulos,constitucion_blancos,constitucion_total,constitucion_validos,convencion_mixta,convencion_constitucional,convencion_nulos,convencion_blancos,convencion_total,convencion_validos
0,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,1.0,NaN,179.0,178,28.0,141.0,6.0,4.0,179.0,169
1,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,1.0,0.0,188.0,187,36.0,142.0,8.0,2.0,188.0,178
2,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,0.0,0.0,188.0,188,30.0,139.0,17.0,2.0,188.0,169
3,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,2.0,NaN,191.0,189,38.0,134.0,14.0,5.0,191.0,172
4,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,2.0,0.0,188.0,186,38.0,137.0,12.0,1.0,188.0,175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44692,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,0.0,0.0,143.0,143,30.0,107.0,3.0,3.0,143.0,137
44693,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,3.0,2.0,132.0,127,29.0,86.0,13.0,4.0,132.0,115
44694,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,1.0,NaN,129.0,128,29.0,96.0,4.0,NaN,129.0,125
44695,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,NaN,NaN,136.0,136,18.0,109.0,7.0,2.0,136.0,127


In [4]:
df.fillna(0)

,region_id,region_name,circ_senatorial_id,circ_senatorial_name,distrito_id,distrito_name,comuna_id,comuna_name,circ_electoral_id,circ_electoral_name,...,constitucion_nulos,constitucion_blancos,constitucion_total,constitucion_validos,convencion_mixta,convencion_constitucional,convencion_nulos,convencion_blancos,convencion_total,convencion_validos
0,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,1.0,0.0,179.0,178,28.0,141.0,6.0,4.0,179.0,169
1,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,1.0,0.0,188.0,187,36.0,142.0,8.0,2.0,188.0,178
2,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,0.0,0.0,188.0,188,30.0,139.0,17.0,2.0,188.0,169
3,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,2.0,0.0,191.0,189,38.0,134.0,14.0,5.0,191.0,172
4,3015,DE ARICA Y PARINACOTA,5001,CIRCUNSCRIPCION SENATORIAL 1,6001,DISTRITO 1,2822,ARICA,7001,ARICA,...,2.0,0.0,188.0,186,38.0,137.0,12.0,1.0,188.0,175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44692,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,0.0,0.0,143.0,143,30.0,107.0,3.0,3.0,143.0,137
44693,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,3.0,2.0,132.0,127,29.0,86.0,13.0,4.0,132.0,115
44694,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,1.0,0.0,129.0,128,29.0,96.0,4.0,0.0,129.0,125
44695,3012,DE MAGALLANES Y DE LA ANTARTICA CHILENA,5015,CIRCUNSCRIPCION SENATORIAL 15,6028,DISTRITO 28,2757,TORRES DEL PAINE,7504,TORRES DEL PAINE (C. CASTILLO),...,0.0,0.0,136.0,136,18.0,109.0,7.0,2.0,136.0,127


In [7]:
df.sum()

region_id                                                            134520107
region_name                  DE ARICA Y PARINACOTADE ARICA Y PARINACOTADE A...
circ_senatorial_id                                                   223840853
circ_senatorial_name         CIRCUNSCRIPCION SENATORIAL 1CIRCUNSCRIPCION SE...
distrito_id                                                          268773456
distrito_name                DISTRITO 1DISTRITO 1DISTRITO 1DISTRITO 1DISTRI...
comuna_id                                                            120253069
comuna_name                  ARICAARICAARICAARICAARICAARICAARICAARICAARICAA...
circ_electoral_id                                                    324678210
circ_electoral_name          ARICAARICAARICAARICAARICAARICAARICAARICAARICAA...
local_id                                                              67437355
local_name                   COLEGIO DEL ALBACOLEGIO DEL ALBACOLEGIO DEL AL...
mesa_id                                             

In [9]:
df['constitucion_total'].sum()

7568840.0

In [10]:
df['convencion_total'].sum()

7563618.0